In [ ]:
#@title Mount Drive { form-width: "30%" }
from google.colab import drive
drive.mount('/content/drive') ## Be sure to pick the right account


Mounted at /content/drive


In [ ]:
#@title Install & Import Packages { form-width: "30%" }
#!pip3 install pillow
#from PIL import Image
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

#!pip install --upgrade smqtk

In [ ]:
#@title Setup CNN { form-width: "30%" }
# https://engmrk.com/alexnet-implementation-using-keras/
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.models import Model
from matplotlib import pyplot
from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib.pyplot import figure


def getModelAlexNet():
  model = Sequential()

  # 1st Convolutional Layer
  model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
  model.add(Activation('relu'))
  # Max Pooling
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

  # 2nd Convolutional Layer
  model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))
  # Max Pooling
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

  # 3rd Convolutional Layer
  model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))

  # 4th Convolutional Layer
  model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))

  # 5th Convolutional Layer
  model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))
  # Max Pooling
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

  # Passing it to a Fully Connected layer
  model.add(Flatten())
  # 1st Fully Connected Layer
  model.add(Dense(4096, input_shape=(224*224*3,)))
  model.add(Activation('relu'))
  # Add Dropout to prevent overfitting
  model.add(Dropout(0.4))

  # 2nd Fully Connected Layer
  model.add(Dense(4096))
  model.add(Activation('relu'))
  # Add Dropout
  model.add(Dropout(0.4))

  # 3rd Fully Connected Layer
  model.add(Dense(1000))
  model.add(Activation('relu'))
  # Add Dropout
  model.add(Dropout(0.4))

  # Output Layer
  model.add(Dense(17))
  model.add(Activation('softmax'))

  #model.summary()

  # Compile the model and load weights
  model.compile(loss=categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
  model.load_weights('/content/drive/My Drive/Mestrado/alexnet_weights.h5', by_name=True)

  return model

def getFeatureAlexNet(layer, images):
  model = getModelAlexNet();
  # redefine model to output right after the first hidden layer
  # (like creating a new model just for doing that as a prediction)
  model = Model(inputs=model.inputs, outputs=model.layers[layer].output)

  # prepare the image (e.g. scale pixel values for the alexnet) !!!
  #img = preprocess_input(images)
  # get feature map for first hidden layer
  features = model.predict(images)

  logger.debug('[getFeatureAlexNet] out: ' + str(features.shape))
  return features

In [ ]:
#@title Load Input Frames { form-width: "30%" }

def loadInput():
  # load the image with the required shape (in this case its 224,224 for alexnet)
  img = load_img('/content/drive/My Drive/Mestrado/UCSDped2/Test/Test012/180.tif', target_size=(224, 224)) #240,360 #224,224
  # convert the image to an array
  img = img_to_array(img)
  # expand dimensions so that it represents a single 'sample'
  img = expand_dims(img, axis=0)

  logger.debug('[loadInput] out: ' + str(img.shape))
  return img

In [ ]:
#@title Extract Feature Maps { form-width: "30%" }

img = loadInput()
frames = img.shape[0]

## The first run of this will always fail ##
feats = getFeatureAlexNet(10, img) # 20 is fc8, 17 is fc7, 10 is conv6
print(img.shape)
print(feats.shape)
rowsFeats = (feats[0].shape)[0]
colsFeats = (feats[0][0].shape)[0]


(1, 224, 224, 3)
(1, 2, 2, 256)


In [ ]:
#@title ITQ { form-width: "30%" }
from collections import Sequence
import sys
import numpy as np
import pdb
# from smqtk.representation.descriptor_element._io import elements_to_matrix


def _norm_vector(v):
    n = np.linalg.norm(v, 2, v.ndim - 1, keepdims=True)
    # replace 0's with 1's, preventing div-by-zero
    n[n == 0.] = 1.

    logger.debug('[_norm_vector] out: ' + str((v / n).shape))
    return v / n
    # Normalization off
    #return v

# Fit the ITQ model given the input set of descriptors. (compress ITQ)
def prepareITQ(x, bit_length):
  # logger.info("Creating matrix of descriptors for fitting")
  # x = elements_to_matrix(descriptors)
  logger.info("descriptor matrix shape: %s", x.shape)

  logger.info("Info normalizing descriptors by factor: fro")
  x = _norm_vector(x)
  norm_x = x

  ## compress ITQ starts here
  logger.info("Centering data")
  mean_vec = np.mean(x, axis=0)
  x -= mean_vec # (X - repmat(mean_vec,size(X,1),1))

  logger.info("Computing PCA transformation")
  logger.info("-- computing covariance")
  # ``cov`` wants each row to be a feature and each column an observation
  # of those features. Thus, each column should be a descriptor vector,
  # thus we need the transpose here.
  c = np.cov(x.transpose())
  # c = np.cov(x)

  # Direct translation from UNC matlab code
  # - eigen vectors are the columns of ``pc``
  logger.info('-- computing linalg.eig')
  l, pc = np.linalg.eig(c)
  # [pc, l] = eigs(C, bit)
  logger.info('-- ordering eigen vectors by descending eigen '
                  'value')

  # # Harry translation of original matlab code
  # # - Uses singular values / vectors, not eigen
  # # - singular vectors are the columns of pc
  # logging.info('-- computing linalg.svd')
  # pc, l, _ = numpy.linalg.svd(c)
  # logging.info('-- ordering singular vectors by descending '
  #                 'singular value')

  # Same ordering method for both eig/svd sources.
  l_pc_ordered = sorted(zip(l, pc.transpose()), key=lambda _p: _p[0], reverse=True)

  logger.info("-- top vector extraction")
  # Only keep the top ``bit_length`` vectors after ordering by descending
  # value magnitude.
  # - Transposing vectors back to column-vectors.
  pc_top = np.array([p[1] for p in l_pc_ordered[:bit_length]]).transpose()
  logger.info("-- project centered data by PC matrix")
  v = np.dot(x, pc_top) # XX = X*pc;

  logger.info("Performing ITQ to find optimal rotation")
  c, r = ITQ(v, 50) # MattRb code uses 300 or 100
  # De-adjust rotation with PC vector
  r = np.dot(pc_top, r) #project_mat = pca_mapping * itq_rot_mat ?

  #self.save_model() adds the matrices to a cache in the object

  return c, pc, r, norm_x

# main function for ITQ which finds a rotation of the PCA embedded data
# Input:
#       V: n*c PCA embedded data, n is the number of images and c is the
#       code length
#       n_iter: max number of iterations, 50 is usually enough
# Output:
#       B: n*c binary matrix
#       R: the c*c rotation matrix found by ITQ

def ITQ(v, n_iter):
  # python version can be found on  _find_itq_rotation
  # initialize with a orthogonal random rotation
  bit = np.size(v,1); # size of dimension 2 (or v.shape[1] "Pull num bits from PCA-projected descriptors")
  r = np.random.randn(bit,bit);
  u11, s2, v2 = np.linalg.svd(r);
  #R = U11(:,1:bit); #entire row, first through bit elements inclusive
  r = u11[:, :bit]
  #r = u11[:,0:bit+1]?

  # ITQ to find optimal rotation
  logger.info("ITQ iterations to determine optimal rotation: %d", n_iter)
  for i in range(n_iter):
      z = np.dot(v, r)      
      ux = np.ones(z.shape)*(-1)
      ux[z >= 0] = 1
      c = np.dot(ux.transpose(), v)
      ub, sigma, ua = np.linalg.svd(c)
      r = np.dot(ua, ub.transpose())

  # make B binary

  #z = np.dot(v, r) recalculate z so it doesnt get desync
  #b = np.zeros(z.shape, dtype=np.bool)
  #b[z >= 0] = True
  b = ux # original
  b[b < 0] = 0 # original

  logger.debug('[ITQ] out: ' + str(b.shape) + ',' + str(r.shape))
  return b,r
        

In [ ]:
#@title Convert to Binary Maps { form-width: "30%" }
#logging.basicConfig(stream=sys.stderr, level=logging.ERROR)

def featureVectorToBinVector(feats):
  array = []
  for image in feats:
    for row in image:
      for col in row:
        array.append(col)
  array = np.array(array)

  c, pc, r, norm_x = prepareITQ(array, 7);
  logger.debug('[featureVectorToBinVector] out: ' + str(c.shape))
  return c

binFeats = []
for frame in range(frames):
  frameFeats = featureVectorToBinVector(feats)
  frameFeats = frameFeats.reshape(rowsFeats, colsFeats, 7)
  print(frameFeats.shape)
  binFeats.append(frameFeats)

binFeats = np.array(binFeats)
print(binFeats.shape)


(2, 2, 7)
(1, 2, 2, 7)


In [ ]:
# plot all 4 vectors in an 2x2 squares
width = 2
height = 2
depth = 7
ix = 1
for _ in range(height):
	for _ in range(width):
		for _ in range(depth):
			# specify subplot and turn of axis      
			print(binFeats[0, :, :, :])
		ix += 1
# show the figure
pyplot.show()

[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1. 0.]]]
[[[0. 1. 0. 1. 0. 0. 1.]
  [1. 1. 1. 0. 0. 0. 1.]]

 [[0. 1. 0. 